# Imports

In [1]:
from ebay_delivery_prediction_project import Preprocessing, Visualisation, preprocessing_models
Preprocessing.import_test()

from datetime import datetime

Preprocessing successfully imported.


In [2]:
# data_object = Preprocessing.read_data(rows_to_read = None, columns = ["b2c_c2c"])
# data_object = Preprocessing.read_data(rows_to_read = None)
training_data = Preprocessing.read_data(rows_to_read = 10000)["train"]
print(training_data.columns)

Reading 10000 rows.
Reading all columns.
Index(['b2c_c2c', 'seller_id', 'declared_handling_days',
       'acceptance_scan_timestamp', 'shipment_method_id', 'shipping_fee',
       'carrier_min_estimate', 'carrier_max_estimate', 'item_zip', 'buyer_zip',
       'category_id', 'item_price', 'quantity', 'payment_datetime',
       'delivery_date', 'weight', 'weight_units', 'package_size',
       'record_number'],
      dtype='object')


# EDA

## Target column

The objective is to estimate the total number of calendar days (after payment) it will take to have a purchased item show up at the buyer’s address. This is equivalent to estimating the delivery date using the formula:

payment date (local time) + delivery calendar days = delivery date (local time).

Participants should provide this delivery date in their submissions.

_Now to generate target column_

### Preprocessing the datetime columns

In [3]:
training_data = Preprocessing.parse_datetime_columns(training_data)

## b2c_c2c

Annexure : Defines whether the transaction belongs to a business seller; possible values are B2C
(business to consumer) or C2C (consumer to consumer).

In [4]:
training_data["b2c_c2c"].value_counts() # Need to read full data but only certain columns

B2C    10634413
C2C     4365587
Name: b2c_c2c, dtype: int64